<a href="https://colab.research.google.com/github/itsvishal07/Mobile_Computing/blob/main/Simulate%20and%20analyse%20CDMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np

def generate_walsh_matrix(size):
    if size == 1:
        return np.array([[1]])
    H_prev = generate_walsh_matrix(size // 2)
    H = np.vstack((
        np.hstack((H_prev, H_prev)),
        np.hstack((H_prev, -H_prev))
    ))
    return H

# --- Simulation Parameters ---
num_users = 8
chip_rate = 1000000  # 1 Mbps
spreading_factor = 8
data_rate = chip_rate / spreading_factor
num_bits = 100

# Generate Walsh codes
walsh_matrix = generate_walsh_matrix(spreading_factor)
user_codes = walsh_matrix[:num_users]

# --- User Data Generation and Spreading ---
user_data = np.random.randint(0, 2, size=(num_users, num_bits)) * 2 - 1  # BPSK: -1 or 1
transmitted_signals = []
for i in range(num_users):
    spread_signal = np.repeat(user_data[i], spreading_factor) * np.tile(user_codes[i], num_bits)
    transmitted_signals.append(spread_signal)

# --- Channel Simulation ---
noise_power_db = -10  # Noise power in dB
noise_power_linear = 10**(noise_power_db / 10)
noise = np.sqrt(noise_power_linear) * np.random.randn(num_users, len(transmitted_signals[0]))

received_signal = np.sum(transmitted_signals, axis=0) + np.sum(noise, axis=0)

# --- Receiver Processing ---
decoded_data = np.zeros_like(user_data)
correlation_results = np.zeros((num_users, num_users))
signal_power = np.zeros(num_users)
noise_power_user = np.zeros(num_users)
interference_power = np.zeros(num_users)
snr_db = np.zeros(num_users)

for i in range(num_users):
    # Despreading
    despread_signal = received_signal.reshape(num_bits, spreading_factor) @ user_codes[i]
    decoded_data[i] = np.sign(despread_signal)

    # Calculate correlation with all user codes
    for j in range(num_users):
        tiled_code = np.tile(user_codes[j], (num_bits, 1))  # <-- Fixed here
        correlation = np.mean(received_signal.reshape(num_bits, spreading_factor) * tiled_code, axis=1)
        correlation_results[i, j] = np.mean(np.abs(correlation))

    # Estimate signal power, interference, and noise for each user
    intended_signal = transmitted_signals[i]
    interference = np.sum([transmitted_signals[j] for j in range(num_users) if j != i], axis=0)

    signal_power[i] = np.mean(intended_signal**2)
    interference_power[i] = np.mean(interference**2)
    noise_power_user[i] = noise_power_linear

    if signal_power[i] > 0:
        snr_db[i] = 10 * np.log10(signal_power[i] / (interference_power[i] + noise_power_user[i]))
    else:
        snr_db[i] = -np.inf

# --- Results Display ---
print("--- Simulation Results ---")
print("\nWalsh Code Matrix:")
print(walsh_matrix)

print("\nUser Data (first 5 bits):")
print(user_data[:, :5])

print("\nDecoded Data (first 5 bits):")
print(decoded_data[:, :5])

print("\nCorrelation Matrix (absolute mean):")
print(correlation_results)

print("\nPer User Statistics:")
print(f"{'User':<5} {'Intended':<10} {'Correlation':<12} {'Signal Power':<15} {'Noise Power':<15} {'SNR (dB)':<10}")
for i in range(num_users):
    intended_str = "Yes" if np.array_equal(decoded_data[i], user_data[i]) else "No"
    print(f"{i+1:<5} {intended_str:<10} {correlation_results[i, i]:<12.6f} {signal_power[i]:<15.6f} {noise_power_user[i]:<15.6f} {snr_db[i]:<10.2f}")


--- Simulation Results ---

Walsh Code Matrix:
[[ 1  1  1  1  1  1  1  1]
 [ 1 -1  1 -1  1 -1  1 -1]
 [ 1  1 -1 -1  1  1 -1 -1]
 [ 1 -1 -1  1  1 -1 -1  1]
 [ 1  1  1  1 -1 -1 -1 -1]
 [ 1 -1  1 -1 -1  1 -1  1]
 [ 1  1 -1 -1 -1 -1  1  1]
 [ 1 -1 -1  1 -1  1  1 -1]]

User Data (first 5 bits):
[[ 1 -1 -1 -1  1]
 [ 1 -1 -1  1 -1]
 [ 1  1 -1 -1  1]
 [ 1  1 -1 -1 -1]
 [-1 -1 -1  1 -1]
 [-1  1 -1  1  1]
 [ 1  1 -1  1 -1]
 [ 1 -1  1  1  1]]

Decoded Data (first 5 bits):
[[ 1 -1 -1 -1  1]
 [ 1 -1 -1  1 -1]
 [ 1  1 -1 -1  1]
 [ 1  1 -1 -1 -1]
 [-1 -1 -1  1 -1]
 [-1  1 -1  1  1]
 [ 1  1 -1  1 -1]
 [ 1 -1  1  1  1]]

Correlation Matrix (absolute mean):
[[0.99717562 0.99968712 0.99727996 1.01549248 1.04243764 1.01468726
  1.05302253 0.95918489]
 [0.99717562 0.99968712 0.99727996 1.01549248 1.04243764 1.01468726
  1.05302253 0.95918489]
 [0.99717562 0.99968712 0.99727996 1.01549248 1.04243764 1.01468726
  1.05302253 0.95918489]
 [0.99717562 0.99968712 0.99727996 1.01549248 1.04243764 1.01468726
  1.0